In [12]:
from dotenv import load_dotenv
load_dotenv()

%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [13]:
%%bigquery open_issues_60_days
SELECT
   opened.issues_id
    , opened.stars_repo_id
    , opened.actor_login 
    , opened.html_url
    , opened.opened_created_at
    , closed.closed_created_at
    , TIMESTAMP_DIFF(closed.closed_created_at, opened.opened_created_at, DAY) as length_in_days
    , opened.repo_name
    , opened.stars
    , opened.labels
    , opened.title
    , best_actor_for_repo.actor_login as best_solver
    , DATE_DIFF(CURRENT_DATE(), DATE(opened.opened_created_at), DAY) as days_outstanding
FROM (
  SELECT
      gd.id as issues_id
    , rs.repo_id stars_repo_id
    , payload
    , JSON_EXTRACT_SCALAR(payload, "$.issue.html_url") as html_url
    , LOWER(JSON_EXTRACT_SCALAR(payload, '$.issue.labels[0].name')) as labels
    , LOWER(JSON_EXTRACT_SCALAR(payload, '$.issue.labels[0].name')) as title
    , repo.name as repo_name
    , repo.url
    , actor.login actor_login
    , created_at as opened_created_at
    , stars
  FROM
    `githubarchive.day.201*` gd
  LEFT JOIN 
    `fh-bigquery.github_extracts.repo_stars` rs ON gd.repo.name = rs.repo_name
  WHERE
    type = 'IssuesEvent'
  AND
    JSON_EXTRACT_SCALAR(payload, "$.action") = 'opened'
) opened
LEFT JOIN
(
  SELECT
    gd.id as id
    , JSON_EXTRACT_SCALAR(payload, "$.issue.html_url") as html_url
    , repo.url
    , actor.login as actor_login
    , created_at as closed_created_at
  FROM
  `githubarchive.day.201*` gd
  WHERE
    type = 'IssuesEvent'
  AND
    JSON_EXTRACT_SCALAR(payload, "$.action") = 'closed'
) closed
ON opened.html_url = closed.html_url AND opened.url = closed.url
JOIN (
   SELECT 
      e.*
   FROM
   (
      SELECT
        c.repo_url
      , c.repo_name
      , MAX(c.number_of_issues_closed_by_actor) as number_of_issues_closed_by_actor
      FROM
      (
       SELECT
       repo.url repo_url
        , repo.name as repo_name
        , COUNT(created_at) as number_of_issues_closed_by_actor
       FROM
       `githubarchive.day.201*` gd
       WHERE
       type = 'IssuesEvent'
       AND
       payload LIKE '%"action":"closed"%'
       GROUP BY
       repo.url, repo.name
      ) c
      GROUP BY c.repo_url, c.repo_name
  ) d
  JOIN
    (
      SELECT
      actor.login as actor_login
      , repo.url repo_url
      , repo.name as repo_name
      , COUNT(created_at) as number_of_issues_closed_by_actor
     FROM
      `githubarchive.day.201*` gd
     WHERE
      type = 'IssuesEvent'
     AND
     payload LIKE '%"action":"closed"%'
     GROUP BY actor_login, repo_url, repo_name
    ) e
  ON d.repo_url = e.repo_url AND d.repo_name = e.repo_name and d.number_of_issues_closed_by_actor = e.number_of_issues_closed_by_actor
) best_actor_for_repo
ON opened.repo_name = best_actor_for_repo.repo_name
WHERE 
  opened_created_at < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 60 DAY) 
  AND opened_created_at > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 61 DAY)
  AND closed_created_at IS NULL
  AND opened.opened_created_at is not null
ORDER BY days_outstanding asc, stars desc
LIMIT 16000

,issues_id,stars_repo_id,actor_login,html_url,opened_created_at,closed_created_at,length_in_days,repo_name,stars,labels,title,best_solver,days_outstanding
0,8682049640,1130565.0,exogen,https://github.com/lorenwest/node-config/issue...,2018-12-02 03:05:45+00:00,None,None,lorenwest/node-config,2586.0,None,None,lorenwest,60
1,8682246329,2431044.0,bhylak,https://github.com/commonsense/conceptnet5/iss...,2018-12-02 05:45:51+00:00,None,None,commonsense/conceptnet5,876.0,None,None,rspeer,60
2,8682167223,85393329.0,chrispsn,https://github.com/chrispsn/mesh/issues/60,2018-12-02 04:39:31+00:00,None,None,chrispsn/mesh,837.0,enhancement,enhancement,chrispsn,60
3,8681908130,43610700.0,lordcodes,https://github.com/ziem/android-development-bl...,2018-12-02 01:19:29+00:00,None,None,ziem/android-development-blogs,287.0,None,None,ziem,60
4,8681877910,856461.0,fuerstenau,https://github.com/SimonKagstrom/kcov/issues/279,2018-12-02 00:55:59+00:00,None,None,SimonKagstrom/kcov,187.0,None,None,SimonKagstrom,60
5,8681844476,99935087.0,jonascarpay,https://github.com/jonascarpay/apecs/issues/22,2018-12-02 00:30:29+00:00,None,None,jonascarpay/apecs,141.0,apecs-physics,apecs-physics,jonascarpay,60
6,8681843572,98999026.0,Anis-Vikernes,https://github.com/ravi8x/Barcode-Reader/issue...,2018-12-02 00:29:52+00:00,None,None,ravi8x/Barcode-Reader,105.0,None,None,ravi8x,60
7,8682179598,71623522.0,tayyabsayyad,https://github.com/nurlansu/hugo-sustain/issue...,2018-12-02 04:49:55+00:00,None,None,nurlansu/hugo-sustain,80.0,None,None,nurlansu,60
8,8682167359,44432648.0,tenntenn,https://github.com/tenntenn/gohandson/issues/29,2018-12-02 04:39:32+00:00,None,None,tenntenn/gohandson,73.0,None,None,tenntenn,60
9,8681884115,17684708.0,WavePhenomena,https://github.com/tiehuis/2048-cli/issues/27,2018-12-02 01:01:03+00:00,None,None,tiehuis/2048-cli,71.0,None,None,tiehuis,60


In [14]:
import requests
import json
import os

GITHUB_URL = "https://api.github.com/"

"""
Function to get JSON response from a URL
:params:
    url     string
:return:
            JSON
"""
def __get_json_response(url):
    headers = {'Authorization': 'token ' + os.environ['GITHUB_API_KEY']}
    response = requests.get(url, headers=headers)
    return json.loads(response.text)

"""
Function to add email to a set of emails and set a loop break flag
:params:
    email_set       set         set of all the emails for the user
    email           string      new email to be added
    max_len         integer     maximum number of emails to be extracted
:return:
    email_set       set         set of all the emails for the user
    break_flag      boolean     if max_limit is reached, break_flag is set to True
"""
def __add_email(email_set, email, max_len):
    email_set = email_set | set([email])
    break_flag = (len(email_set) >= max_len)

    return email_set, break_flag

"""
Function to get user emails using GitHub APIs
:params:
    user        string      a valid GitHub username
    max_limit   integer     maximum number of email ID to be fetched
:return:
    user_email  set         a set of all emails extracted
    message     string      if any error occurs, this holds the respective error message
"""
def __get_github_emails(user, max_limit):
    user_email = set([])
    break_flag = False
    try:
        users_profile_url = GITHUB_URL + "users/{0}".format(user)
        response = __get_json_response(users_profile_url)

        # some error encountered
        if 'message' in response:
            if response['message'] == 'Not Found':
                return u'You need to enter a valid GitHub Username'
            else:
                return response['message']
        name = response['name']
        user_name = response['name']
        company = response['company']
        # if user has a public email, add that to the set of emails
        if response['email']:
            user_email, break_flag = __add_email(user_email, response['email'], max_limit)

        if not break_flag:
            users_repository_url = GITHUB_URL + "users/{0}/repos?type=owner&sort=updated".format(user)
            response = __get_json_response(users_repository_url)

            for repo in response:
                if not repo['fork']:
                    users_repository_name = repo['full_name']
                    repos_commit_url = GITHUB_URL + "repos/{0}/commits".format(users_repository_name)
                    commit_reponse = __get_json_response(repos_commit_url)
                    
                    possible_positions = ['committer', 'author']

                    for commit in commit_reponse:
                        for i in possible_positions:
                            if commit['commit'][i]['name'] == user_name:
                                email_string = commit['commit'][i]['email']
                                if "noreply" not in email_string:
                                    user_email, break_flag = __add_email(user_email, email_string, max_limit)

                        if break_flag:
                            break

                if break_flag:
                    break

        if len(user_email) > 0:
            return user_email, name, company
        else:
            return u'No emails found', u'No first name found', u'No company found'

    except requests.exceptions.ConnectionError:
        return u'Proper internet connection not found', u'Proper internet connection not found', u'Proper internet connection not found'

"""
Function to get the emails associated to a username on GitHub
:params:
    username    string      a valid GitHub username
    num         integer     maximum number of email ID to be fetched, default 1
:return:
    response    JSON response
        success     boolean     flag to determine other key in JSON
        email       list        if 'success' is True, list of all the emails fetched
        message     string      if 'success' is False, returns the error message
"""
def get(username, num=1):
    email, name, co = __get_github_emails(username, num)
    if type(email) == set:
        response = {
            'success' : True,
            'data': [email.pop(), name, co]
        }
    else:
        response = {
            'success' : False,
            'message' : [email, name, co]
        }

    return response

In [ ]:
import pandas as pd
result = pd.DataFrame()
for index, row in open_issues_60_days.iterrows():
    print(index)
    issues_id = row['issues_id']
    stars_repo_id = row['stars_repo_id']
    actor_login = row['actor_login']
    html_url = row['html_url']
    opened_created_at = row['opened_created_at']
    closed_created_at = row['closed_created_at']
    length_in_days = row['length_in_days']
    repo_name = row['repo_name']
    stars = row['stars']
    labels = row['labels']
    best_solver = row['best_solver']
    days_outstanding = row['days_outstanding']
    title = row['title']
    try:
        resp = get(actor_login)
        if 'data' in resp:
            email, name, co  =resp['data']
            row['email'] = email
            row['name'] = name
            row['company'] = co
            if ".com" in email:
                result = result.append(row, ignore_index=True)
    except Exception as e:
        print(e)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
string indices must be integers
63
string indices must be integers
64
65
66
67
68
string indices must be integers
69
70
71
72
73
74
75
76
77
78
79
80
81
82
string indices must be integers
83
84
85
86
87
88
89
90
string indices must be integers
91
92
93
94
95
96
97
98
99
100
101
102
103
104
string indices must be integers
105
106
107
108
109
110
111
112


In [8]:

result.to_csv('open_issue_60_days.csv', encoding = 'utf-8')

In [9]:
result

,actor_login,best_solver,closed_created_at,company,days_outstanding,email,html_url,issues_id,labels,length_in_days,name,opened_created_at,repo_name,stars,stars_repo_id
0,exogen,lorenwest,None,@FormidableLabs,60.0,exogen@gmail.com,https://github.com/lorenwest/node-config/issue...,8682049640,None,None,Brian Beck,2018-12-02 03:05:45+00:00,lorenwest/node-config,2586.0,1130565.0
1,chrispsn,chrispsn,None,None,60.0,chrispsn@gmail.com,https://github.com/chrispsn/mesh/issues/60,8682167223,enhancement,None,Chris Pearson,2018-12-02 04:39:31+00:00,chrispsn/mesh,837.0,85393329.0
2,lordcodes,ziem,None,GetBusy,60.0,andrewlord1990@gmail.com,https://github.com/ziem/android-development-bl...,8681908130,None,None,Andrew Lord,2018-12-02 01:19:29+00:00,ziem/android-development-blogs,287.0,43610700.0
3,jonascarpay,jonascarpay,None,TU Delft,60.0,jonascarpay@gmail.com,https://github.com/jonascarpay/apecs/issues/22,8681844476,apecs-physics,None,Jonas Carpay,2018-12-02 00:30:29+00:00,jonascarpay/apecs,141.0,99935087.0
4,tayyabsayyad,nurlansu,None,Don Bosco institute of Technology.,60.0,tayyabsayyad@gmail.com,https://github.com/nurlansu/hugo-sustain/issue...,8682179598,None,None,Tayyabali Sayyad,2018-12-02 04:49:55+00:00,nurlansu/hugo-sustain,80.0,71623522.0
5,tenntenn,tenntenn,None,"Souzoh, Inc. (affiliated by Mercari, Inc.)",60.0,uedatakuya@gmail.com,https://github.com/tenntenn/gohandson/issues/29,8682167359,None,None,Takuya Ueda,2018-12-02 04:39:32+00:00,tenntenn/gohandson,73.0,44432648.0
6,snugghash,brian717,None,None,60.0,suhas.gundimeda@gmail.com,https://github.com/HumanDynamics/openPDS/issue...,8681880940,None,None,Suhas Gundimeda,2018-12-02 00:58:37+00:00,HumanDynamics/openPDS,48.0,9619735.0
7,ajoberstar,ajoberstar,None,None,60.0,ajoberstar@gmail.com,https://github.com/gradle-clojure/gradle-cloju...,8681976948,None,None,Andrew Oberstar,2018-12-02 02:11:14+00:00,gradle-clojure/gradle-clojure,27.0,99865757.0
8,tyleransom,tyleransom,None,University of Oklahoma,60.0,tyleransom@gmail.com,https://github.com/tyleransom/DScourseS18/issu...,8682109507,None,None,Tyler Ransom,2018-12-02 03:52:09+00:00,tyleransom/DScourseS18,25.0,106866087.0
9,mlenzen,mlenzen,None,None,60.0,m.lenzen@gmail.com,https://github.com/mlenzen/collections-extende...,8681901159,bug,None,Michael Lenzen,2018-12-02 01:14:09+00:00,mlenzen/collections-extended,17.0,12181352.0


In [42]:
!open top_1000_github_users.csv

In [39]:
result[result.columns]

,city,company,email,fake,follower_count,follower_id,login,name,state
0,Portland,Linux Foundation,torvalds@linux-foundation.org,0.0,80184.0,5203.0,torvalds,Linus Torvalds,OR
1,Pittsburgh,"Google, Inc.",jakewharton@gmail.com,0.0,48120.0,896.0,JakeWharton,Jake Wharton,PA
2,Shanghai,None,yifeng.ruan@gmail.com,0.0,39102.0,616741.0,ruanyf,Ruan YiFeng,None
3,Victoria,Apex,tj@apex.sh,0.0,37402.0,376498.0,Tj,TJ Holowaychuk,BC
4,Mountain View,Google,addyosmani@gmail.com,0.0,32666.0,6240.0,addyosmani,Addy Osmani,CA
5,Palo Alto,"Google Chrome, ♥z",paul.irish@gmail.com,0.0,29690.0,1779.0,paulirish,Paul Irish,CA
6,None,None,yyx990803@gmail.com,0.0,29200.0,18556.0,yyx990803,Evan You,None
7,London,@facebook,dan.abramov@me.com,0.0,27415.0,417948.0,gaearon,Dan Abramov,England
8,None,@avajs @xojs @chalk,sindresorhus@gmail.com,0.0,25701.0,3871.0,sindresorhus,Sindre Sorhus,None
9,San Francisco,None,tom@mojombo.com,0.0,25112.0,9236.0,mojombo,Tom Preston-Werner,CA
